In [1]:
import mnist_data
import tensorflow as tf
import tensorflowvisu
import math
tf.set_random_seed(0)

mnist = mnist_data.read_data_sets("mydata")

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y_ = tf.placeholder(tf.float32, [None, 10])
lr = tf.placeholder(tf.float32)
pkeep = tf.placeholder(tf.float32)

Loading mydata/train-images-idx3-ubyte.mnist
Loading mydata/train-labels-idx1-ubyte.mnist
Loading mydata/t10k-images-idx3-ubyte.mnist
Loading mydata/t10k-labels-idx1-ubyte.mnist


In [7]:
type(mnist.train)

mnist_data.DataSet

In [2]:
def create_nn_model():
    # three convolutional layers with their channel counts, and a
    # fully connected layer (tha last layer has 10 softmax neurons)
    K = 6  # first convolutional layer output depth
    L = 12  # second convolutional layer output depth
    M = 24  # third convolutional layer
    N = 200  # fully connected layer

    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1))  # 6x6 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]))
    W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1))
    B2 = tf.Variable(tf.constant(0.1, tf.float32, [L]))
    W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1))
    B3 = tf.Variable(tf.constant(0.1, tf.float32, [M]))

    W4 = tf.Variable(tf.truncated_normal([7 * 7 * M, N], stddev=0.1))
    B4 = tf.Variable(tf.constant(0.1, tf.float32, [N]))
    W5 = tf.Variable(tf.truncated_normal([N, 10], stddev=0.1))
    B5 = tf.Variable(tf.constant(0.1, tf.float32, [10]))

    # The model
    stride = 1  # output is 28x28
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    stride = 2  # output is 14x14
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    stride = 2  # output is 7x7
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

    # reshape the output from the third convolution for the fully connected layer
    YY = tf.reshape(Y3, shape=[-1, 7 * 7 * M])

    Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
    YY4 = tf.nn.dropout(Y4, pkeep)
    Ylogits = tf.matmul(YY4, W5) + B5

    return Ylogits

In [5]:
predictions  = create_nn_model()
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(predictions, Y_))*100
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

In [21]:
foo  = mnist.test.images
foo0 = foo[0]

In [48]:
type(foo0)

numpy.ndarray

In [50]:
foo.shape

(10000, 28, 28, 1)

In [85]:
import struct
from matplotlib import pyplot as plt

In [2]:
#http://feltes.ch/index.php/2015/09/20/mnist-handwritten-digits-database-decoding-with-python/
def read_images(images_name):
    #returns an array of flattened images
    f = open(images_name, "rb")
    ds_images = []
    #Let's read the head of the file encoded in 32-bit integers in big-endian(4 bytes)
    mw_32bit = f.read(4)        #magic word
    n_numbers_32bit = f.read(4) #number of images
    n_rows_32bit = f.read(4)    #number of rows of each image
    n_columns_32bit = f.read(4) #number of columns of each image
    
    #convert it to integers ; '>i' for big endian encoding
    mw =  struct.unpack('>i',mw_32bit)[0] 
    n_numbers = struct.unpack('>i',n_numbers_32bit)[0] 
    n_rows = struct.unpack('>i',n_rows_32bit)[0]
    n_columns = struct.unpack('>i',n_columns_32bit)[0]
    
    try:
        for i in range(n_numbers):
            image = []
            for r in range(n_rows):
                for l in range(n_columns):
                    byte = f.read(1)
                    pixel = struct.unpack('B',byte[0])[0]
                    image.append(pixel)            
            ds_images.append(image)
        
    finally:
        f.close()
    return ds_images

In [3]:
def read_labels(labels_name):
    #returns an array of labels
    f = open(labels_name, "rb")
    ds_labels = []
    #Let's read the head of the file encoded in 32-bit integers in big-endian(4 bytes)
    mw_32bit = f.read(4)        #magic word
    n_numbers_32bit = f.read(4) #number of labels
    
    #convert it to integers ; '>i' for big endian encoding
    mw =  struct.unpack('>i',mw_32bit)[0] 
    n_numbers = struct.unpack('>i',n_numbers_32bit)[0] 
    
    try:
        for i in range(n_numbers):
            byte = f.read(1)
            label = struct.unpack('B',byte[0])[0]      
            ds_labels.append(label)
        
    finally:
        f.close()
    return ds_labels

In [28]:
import struct
import numpy
from matplotlib import pyplot as plt
import pylab
def read_dataset(images_name,labels_name):
    #reads an image-file and a labels file, and returns an array of tuples of
    #(flattened_image, label)
    images = read_images(images_name)
    labels = read_labels(labels_name)
    assert len(images) == len(labels)
    return zip(images,labels)

In [17]:
testset = read_dataset("mydata/t10k-images-idx3-ubyte.mnist","mydata/t10k-labels-idx1-ubyte.mnist")
#read_images("t10k-images.idx3-ubyte")
#trainingset = read_dataset("train-images.idx3-ubyte","train-labels.idx1-ubyte")


In [22]:
dataset = testset

In [29]:
fig = plt.figure(figsize=(10,20))
for i in range(50):
    sp = fig.add_subplot(10,5,i+1)
    sp.set_title(dataset[i][1])
    plt.axis('off')
    image = numpy.array(dataset[i][0]).reshape(28,28)
    plt.imshow(image,interpolation='none',cmap=pylab.gray(),label=dataset[i][1])
    
plt.show()